In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as scpy
from scipy import stats
from scipy import ndimage
from scipy.interpolate import griddata
from skimage.transform import resize


import dionysus as d

import os
import csv
import pandas as pd
import seaborn as sns
import pickle

from sklearn.cluster import KMeans

from mpl_toolkits.mplot3d import Axes3D
from skimage import io

import importlib
import helper_functions
importlib.reload(helper_functions)
from helper_functions import *

# %matplotlib notebook
import warnings
warnings.filterwarnings('ignore')

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'arial'
# rcParams['font.sans-serif'] = ['Tahoma']
rcParams.update({'font.size': 16})

In [2]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

def max_min_scaled(dist):
    return((dist-np.min(dist))/(np.max(dist)-np.min(dist)))

In [4]:
average_radial = pickle.load(open('RadialAveragingPodosome0to600nm.p','rb'))

In [6]:
pix_size = 0.01

nr = len(average_radial.T)
nh = len(average_radial)
ntheta = 102
max_r = len(average_radial.T)*pix_size*1000
r,h,t = np.ogrid[0:max_r:nr+0j, 0:450:nh+0j,0:2*np.pi:ntheta+0j]


In [11]:
rs = np.linspace(0,max_r,len(average_radial.T))
hs = np.linspace(0,10*(len(average_radial)-1),len(average_radial))
nthetas = 360+1
thetas = np.radians(np.linspace(0,360,nthetas))
average_radial_polar = np.dstack([average_radial]*nthetas)

In [8]:
# Polar meshgrid
rad_c, theta_c = np.meshgrid(rs, thetas)

# Cartesian equivalents of polar co-ordinates
X = rad_c*np.cos(theta_c)
Y = rad_c*np.sin(theta_c)

# Cartesian (x/y) meshgrid
grid_spacing = 8.0   # You can change this
nx = (X.max() - X.min())/grid_spacing
ny = (Y.max() - Y.min())/grid_spacing
x = np.arange(X.min(), X.max() + grid_spacing, grid_spacing)
y = np.arange(Y.min(), Y.max() + grid_spacing, grid_spacing)
grid_x, grid_y = np.meshgrid(x, y)

# Interpolate from polar to cartesian grid
new_grid = griddata(
    (X.flatten(), Y.flatten()),
    average_radial_polar[40].T.flatten(),
    (grid_x, grid_y),
    method='cubic'
)



In [9]:
stacked_pod_object = []
for i in range(len(average_radial_polar)):
    new_grid = griddata(
    (X.flatten(), Y.flatten()),
    average_radial_polar[i].T.flatten(),
    (grid_x, grid_y),
    method='cubic')
    
    stacked_pod_object.append(new_grid)

In [12]:
stacked_pod_object = np.array(stacked_pod_object)
stacked_pod_object = np.nan_to_num(stacked_pod_object)
stacked_pod_object = max_min_scaled(stacked_pod_object)

In [23]:
pickle.dump(max_min_scaled(scpy.ndimage.gaussian_filter(stacked_pod_object,2)), open( "stacked_pod_object_cart_slice.p", "wb" ) )
